In [ ]:
#talk about hw, https://www.quora.com/Why-is-AUC-Area-under-ROC-insensitive-to-class-distribution-changes 
#https://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/
#ensemble,
#https://www.quora.com/What-will-happen-to-AUC-if-I-switch-the-positive-and-negative-classes-in-the-test-data
#https://medium.com/coinmonks/practical-tips-for-class-imbalance-in-binary-classification-6ee29bcdb8a7

#https://towardsdatascience.com/fine-tuning-a-classifier-in-scikit-learn-66e048c21e65
#https://stats.stackexchange.com/questions/235808/binary-classification-with-strongly-unbalanced-classes
#https://pdfs.semanticscholar.org/9908/404807bf6b63e05e5345f02bcb23cc739ebd.pdf

In [103]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import pandas as pd
import xgboost as xgb
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, auc, roc_curve, f1_score,  confusion_matrix, roc_auc_score
from imblearn.over_sampling import SMOTE
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler


# from termcolor import colored
from xgboost import plot_importance
#install xgboost on mac:
# brew install gcc@7 
# pip install xgboost

#https://github.com/jmwdpk/Dont-Get-Kicked/blob/master/dontgetkickedKaggle.ipynb
#https://www.kaggle.com/tilii7/hyperparameter-grid-search-with-xgboost

# pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# Import and store dataset
fraud_data = pd.read_csv('Fraud_Data.csv')
# fraud_data = pd.read_csv('imbalancedFraudDF.csv')

# ipToCountry = pd.read_csv('IpAddress_to_Country.csv')

In [2]:
fraud_data.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0
2,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0
3,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0
4,159135,2015-05-21 06:03:03,2015-07-09 08:05:14,42,ALEYXFXINSXLZ,Ads,Chrome,M,18,2.809315e+09,0


In [ ]:
data[data['Games Owned'] > 20]

In [104]:
badAll = fraud_data[fraud_data['class'] == 1]
goodAll = fraud_data[fraud_data['class'] == 0]
bad = badAll.sample(frac=0.1, replace=False, random_state = 1)
bad2 = badAll.sample(frac=0.06, replace=False, random_state = 1)

In [105]:
newFraudDF = pd.concat([goodAll, bad])
newFraudDF2 = pd.concat([goodAll, bad2])
newFraudDF['class'].value_counts()##fraud 1.02%

0    136961
1      1415
Name: class, dtype: int64

In [106]:
newFraudDF.to_csv("imbalancedFraudDF.csv", index=False)
newFraudDF2.to_csv("imbalancedFraudDF2.csv", index=False)
newFraudDF2['class'].value_counts()##fraud 1.02%

0    136961
1       849
Name: class, dtype: int64

In [107]:
849*1.0/(849+136961)

0.006160655975618605

In [3]:
#old:
fraud_data['class'].value_counts()
#fraud 9.36%

0    136961
1     14151
Name: class, dtype: int64

In [2]:
fraud_data['class'].value_counts()

0    136961
1      1415
Name: class, dtype: int64

In [4]:
fraud_data.dtypes

user_id             int64
signup_time        object
purchase_time      object
purchase_value      int64
device_id          object
source             object
browser            object
sex                object
age                 int64
ip_address        float64
class               int64
dtype: object

In [40]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [3]:
import time
start = time.time()



countries = []
for i in range(len(fraud_data)):
    ip_address = fraud_data.loc[i, 'ip_address']
    tmp = ipToCountry[(ipToCountry['lower_bound_ip_address'] <= ip_address) &
                    (ipToCountry['upper_bound_ip_address'] >= ip_address)]
    if len(tmp) == 1:
        countries.append(tmp['country'].values[0])
    else:
        countries.append('NA')
        
fraud_data['country'] = countries
print "Lookup took", time.time() - start, "seconds."
#fraud_data.head()


Lookup took 209.881716967 seconds.


In [4]:
fraud_data.to_csv("imbalancedFraudDFwithCountry.csv", index=False)


# Q: How to check if column user_id is unique(no dup)?


In [71]:
print fraud_data2.user_id.nunique()#138289
print fraud_data2.count()#138376
#most of the user_id has only 1 transaction, difficult to do time based aggregates, e.g. amount in past 1 day for the user

138289


In [41]:
fraud_data = pd.read_csv('imbalancedFraudDFwithCountry.csv')

## Feature Engineering

In [42]:
#time related features: can be done before split, as they has no interaction between other rows
fraud_data['interval_after_signup'] = (pd.to_datetime(fraud_data['purchase_time']) - pd.to_datetime(
        fraud_data['signup_time'])).dt.total_seconds()

fraud_data['signup_days_of_year'] = pd.DatetimeIndex(fraud_data['signup_time']).dayofyear

fraud_data['signup_seconds_of_day'] = pd.DatetimeIndex(fraud_data['signup_time']).second + 60 * pd.DatetimeIndex(
    fraud_data['signup_time']).minute + 3600 * pd.DatetimeIndex(fraud_data['signup_time']).hour

fraud_data['purchase_days_of_year'] = pd.DatetimeIndex(fraud_data['purchase_time']).dayofyear
fraud_data['purchase_seconds_of_day'] = pd.DatetimeIndex(fraud_data['purchase_time']).second + 60 * pd.DatetimeIndex(
    fraud_data['purchase_time']).minute + 3600 * pd.DatetimeIndex(fraud_data['purchase_time']).hour



In [33]:
fraud_data.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,country,interval_after_signup,signup_days_of_year,signup_seconds_of_day,purchase_days_of_year,purchase_seconds_of_day
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,Japan,4506682.0,55,82549,108,10031
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,United States,17944.0,158,74390,159,5934
2,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0,NaN,492085.0,118,76405,124,50090
3,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,United States,4361461.0,202,25792,252,67253
4,159135,2015-05-21 06:03:03,2015-07-09 08:05:14,42,ALEYXFXINSXLZ,Ads,Chrome,M,18,2.809315e+09,0,Canada,4240931.0,141,21783,190,29114


In [43]:
# fraudDF = fraud_data.drop(['user_id','signup_time',
#  'purchase_time','device_id','ip_address','country',
# ], axis=1)


fraud_data = pd.get_dummies(fraud_data, columns=['source', 'browser'])#need to drop ['source', 'browser']? no, auto dropped by get_dummies 
fraud_data['sex'] = (fraud_data.sex == 'M').astype(int)

fraudDF = fraud_data.drop(['user_id','signup_time','purchase_time'], axis=1)
fraudDF.apply(lambda x: x.count(), axis=0)

y = fraudDF['class']
X = fraudDF.drop(['class'], axis=1)



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print("X_train.shape:", X_train.shape)
print("y_train.shape:", y_train.shape)
fraudDF.head()

('X_train.shape:', (110700, 19))
('y_train.shape:', (110700,))


,purchase_value,device_id,sex,age,ip_address,class,country,interval_after_signup,signup_days_of_year,signup_seconds_of_day,purchase_days_of_year,purchase_seconds_of_day,source_Ads,source_Direct,source_SEO,browser_Chrome,browser_FireFox,browser_IE,browser_Opera,browser_Safari
0,34,QVPSPJUOCKZAR,1,39,7.327584e+08,0,Japan,4506682.0,55,82549,108,10031,0,0,1,1,0,0,0,0
1,16,EOGFQPIZPYXFZ,0,53,3.503114e+08,0,United States,17944.0,158,74390,159,5934,1,0,0,1,0,0,0,0
2,44,ATGTXKYKUDUQN,1,41,3.840542e+09,0,NaN,492085.0,118,76405,124,50090,0,0,1,0,0,0,0,1
3,39,NAUITBZFJKHWW,1,45,4.155831e+08,0,United States,4361461.0,202,25792,252,67253,1,0,0,0,0,0,0,1
4,42,ALEYXFXINSXLZ,1,18,2.809315e+09,0,Canada,4240931.0,141,21783,190,29114,1,0,0,1,0,0,0,0


In [44]:
fraudDF.to_csv("imbalancedFraudDFwithCountryNoIDtime.csv", index=False)


In [15]:
X_train.head()

,purchase_value,device_id,source,browser,sex,age,ip_address,country,interval_after_signup,signup_days_of_year,signup_seconds_of_day,purchase_days_of_year,purchase_seconds_of_day
29343,12,OULPAZAFRFPXP,Ads,Chrome,M,42,3.690922e+09,Korea Republic of,3499664.0,183,67384,224,24648
12190,10,AIIWMFEYQQIEB,Ads,Opera,M,29,1.686759e+09,United States,6766039.0,5,78146,84,18585
19388,34,VUVETBUPCIWJE,Direct,Chrome,M,53,4.138429e+09,NaN,5870515.0,197,81354,265,76669
89104,48,QCFULAJOYKFUU,Ads,Chrome,M,29,9.617337e+07,France,2145618.0,160,30920,185,16538
82082,44,IHRWLMIJMEEEU,Ads,FireFox,M,24,1.936025e+09,China,7079059.0,111,71897,193,66156


In [45]:
#need to be done after split, and needs to be standalized/normalized, otherwise, 
#train count range for x: [1000, 5000] count, while test range [100, 500], different distribution :(

# the more a device is shared, the more suspicious
X_train['n_dev_shared'] = X_train.device_id.map(X_train.device_id.value_counts(dropna=False))#without dropna=False will produce nan in this col

# the more a ip is shared, the more suspicious
X_train['n_ip_shared'] = X_train.ip_address.map(X_train.ip_address.value_counts(dropna=False))

# the less visit from a country, the more suspicious
X_train['n_country_shared'] = X_train.country.map(X_train.country.value_counts(dropna=False))

X_train = X_train.drop(['device_id','ip_address','country'], axis=1)


# the more a device is shared, the more suspicious
X_test['n_dev_shared'] = X_test.device_id.map(X_test.device_id.value_counts(dropna=False))

# the more a ip is shared, the more suspicious
X_test['n_ip_shared'] = X_test.ip_address.map(X_test.ip_address.value_counts(dropna=False))

# the less visit from a country, the more suspicious
X_test['n_country_shared'] = X_test.country.map(X_test.country.value_counts(dropna=False))

X_test = X_test.drop(['device_id','ip_address','country'], axis=1)


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vi

In [38]:
# the more a device is shared, the more suspicious
X_train.n_dev_shared.value_counts(dropna=False)#without dropna=False will produce nan in this col



1    105373
2      4756
3       375
4       132
5        40
6        24
Name: n_dev_shared, dtype: int64

In [46]:
# https://medium.com/@rrfd/standardize-or-normalize-examples-in-python-e3f174b65dfc
# normalize to [0,1], standardize to normal, mu=0, can < 0, so we do normalize here
#https://stackoverflow.com/questions/50565937/how-to-normalize-the-train-and-test-data-using-minmaxscaler-sklearn

min_max_scaler = preprocessing.MinMaxScaler()
#fit and transform the training data and use them for the model training
X_train[['n_dev_shared', 'n_ip_shared', 'n_country_shared']] = min_max_scaler.fit_transform(X_train[['n_dev_shared', 'n_ip_shared', 'n_country_shared']])

#fit the model
# model.fit(X_train[['n_dev_shared', 'n_ip_shared', 'n_country_shared']])

#after the model training on the transformed training data define the testing data df_test

#before the prediction of the test data, ONLY APPLY the scaler on them
X_test[['n_dev_shared', 'n_ip_shared', 'n_country_shared']] = min_max_scaler.fit_transform(X_test[['n_dev_shared', 'n_ip_shared', 'n_country_shared']])

#test the model
# y_predicted_from_model = model.predict(X_test[['n_dev_shared', 'n_ip_shared', 'n_country_shared']])



In [39]:

X_train.n_dev_shared.value_counts(dropna=False)




0.0    105373
0.2      4756
0.4       375
0.6       132
0.8        40
1.0        24
Name: n_dev_shared, dtype: int64

In [40]:
X_test.n_dev_shared.value_counts(dropna=False)

0.000000    27310
0.333333      350
0.666667       12
1.000000        4
Name: n_dev_shared, dtype: int64

In [8]:

fraud_data.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,sex,age,ip_address,class,country,interval_after_signup,signup_days_of_year,signup_seconds_of_day,purchase_days_of_year,purchase_seconds_of_day,n_dev_shared,n_ip_shared,n_country_shared,source_Ads,source_Direct,source_SEO,browser_Chrome,browser_FireFox,browser_IE,browser_Opera,browser_Safari
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,1,39,7.327584e+08,0,Japan,4506682.0,55,10069,108,10031,1,1,6664,0,0,1,1,0,0,0,0
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,0,53,3.503114e+08,0,United States,17944.0,158,5930,159,5934,1,1,53071,1,0,0,1,0,0,0,0
2,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,1,41,3.840542e+09,0,NA,492085.0,118,50065,124,50090,1,1,20280,0,0,1,0,0,0,0,1
3,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,1,45,4.155831e+08,0,United States,4361461.0,202,67252,252,67253,1,1,53071,1,0,0,0,0,0,0,1
4,159135,2015-05-21 06:03:03,2015-07-09 08:05:14,42,ALEYXFXINSXLZ,1,18,2.809315e+09,0,Canada,4240931.0,141,29103,190,29114,1,1,2659,1,0,0,1,0,0,0,0


In [7]:
print(fraud_data.country.value_counts(), '\n')

(United States                      58049
NA                                 21966
China                              12038
Japan                               7306
United Kingdom                      4490
Korea Republic of                   4162
Germany                             3646
France                              3161
Canada                              2975
Brazil                              2961
Italy                               1944
Australia                           1844
Netherlands                         1680
Russian Federation                  1616
India                               1310
Taiwan; Republic of China (ROC)     1237
Mexico                              1121
Sweden                              1090
Spain                               1027
South Africa                         838
Switzerland                          785
Poland                               729
Argentina                            661
Indonesia                            649
Norway         

In [25]:
fraud_data['class'].value_counts()

0    136961
1      1415
Name: class, dtype: int64

In [10]:
print(fraud_data.sex.value_counts(), '\n')

(M    88293
F    62819
Name: sex, dtype: int64, '\n')


In [9]:
print(fraud_data.browser.value_counts(), '\n')

(Chrome     61432
IE         36727
Safari     24667
FireFox    24610
Opera       3676
Name: browser, dtype: int64, '\n')


In [8]:
print(fraud_data.source.value_counts(), '\n')


(SEO       60615
Ads       59881
Direct    30616
Name: source, dtype: int64, '\n')


In [13]:
pd.set_option('display.max_columns', 500)

fraud_data.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,sex,age,ip_address,class,country,interval_after_signup,signup_days_of_year,signup_seconds_of_day,purchase_days_of_year,purchase_seconds_of_day,n_dev_shared,n_ip_shared,n_country_shared,source_Ads,source_Direct,source_SEO,browser_Chrome,browser_FireFox,browser_IE,browser_Opera,browser_Safari
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,1,39,7.327584e+08,0,Japan,4506682.0,55,10069,108,10031,1,1,6664,0,0,1,1,0,0,0,0
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,0,53,3.503114e+08,0,United States,17944.0,158,5930,159,5934,1,1,53071,1,0,0,1,0,0,0,0
2,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,1,41,3.840542e+09,0,NA,492085.0,118,50065,124,50090,1,1,20280,0,0,1,0,0,0,0,1
3,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,1,45,4.155831e+08,0,United States,4361461.0,202,67252,252,67253,1,1,53071,1,0,0,0,0,0,0,1
4,159135,2015-05-21 06:03:03,2015-07-09 08:05:14,42,ALEYXFXINSXLZ,1,18,2.809315e+09,0,Canada,4240931.0,141,29103,190,29114,1,1,2659,1,0,0,1,0,0,0,0


In [9]:
# features = ['purchase_value', 'sex', 'age','interval_after_signup','signup_days_of_year',
#  'signup_seconds_of_day',
#  'purchase_days_of_year',
#  'purchase_seconds_of_day',
#  'n_dev_shared',
#  'n_ip_shared',
#  'n_country_shared']

In [6]:
list(fraud_data.head(0))

['user_id',
 'signup_time',
 'purchase_time',
 'purchase_value',
 'device_id',
 'sex',
 'age',
 'ip_address',
 'class',
 'country',
 'interval_after_signup',
 'signup_days_of_year',
 'signup_seconds_of_day',
 'purchase_days_of_year',
 'purchase_seconds_of_day',
 'n_dev_shared',
 'n_ip_shared',
 'n_country_shared',
 'source_Ads',
 'source_Direct',
 'source_SEO',
 'browser_Chrome',
 'browser_FireFox',
 'browser_IE',
 'browser_Opera',
 'browser_Safari']

In [7]:
fraud_data.dtypes

user_id                      int64
signup_time                 object
purchase_time               object
purchase_value               int64
device_id                   object
sex                          int64
age                          int64
ip_address                 float64
class                        int64
country                     object
interval_after_signup      float64
signup_days_of_year          int64
signup_seconds_of_day        int64
purchase_days_of_year        int64
purchase_seconds_of_day      int64
n_dev_shared                 int64
n_ip_shared                  int64
n_country_shared             int64
source_Ads                   uint8
source_Direct                uint8
source_SEO                   uint8
browser_Chrome               uint8
browser_FireFox              uint8
browser_IE                   uint8
browser_Opera                uint8
browser_Safari               uint8
dtype: object

In [47]:
# fraudDF = fraud_data.drop(['user_id','signup_time',
#  'purchase_time','device_id','ip_address','country',
# ], axis=1)
fraudDF.apply(lambda x: x.count(), axis=0)

purchase_value             138376
device_id                  138376
sex                        138376
age                        138376
ip_address                 138376
class                      138376
country                    118096
interval_after_signup      138376
signup_days_of_year        138376
signup_seconds_of_day      138376
purchase_days_of_year      138376
purchase_seconds_of_day    138376
source_Ads                 138376
source_Direct              138376
source_SEO                 138376
browser_Chrome             138376
browser_FireFox            138376
browser_IE                 138376
browser_Opera              138376
browser_Safari             138376
dtype: int64

In [48]:
fraudDF.dtypes

purchase_value               int64
device_id                   object
sex                          int64
age                          int64
ip_address                 float64
class                        int64
country                     object
interval_after_signup      float64
signup_days_of_year          int64
signup_seconds_of_day        int64
purchase_days_of_year        int64
purchase_seconds_of_day      int64
source_Ads                   uint8
source_Direct                uint8
source_SEO                   uint8
browser_Chrome               uint8
browser_FireFox              uint8
browser_IE                   uint8
browser_Opera                uint8
browser_Safari               uint8
dtype: object

('X_train.shape:', (110700, 19))
('y_train.shape:', (110700,))


In [51]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, auc, roc_curve, f1_score,  confusion_matrix, roc_auc_score


classifier_RF = RandomForestClassifier(random_state=0, max_depth=3)

classifier_RF.fit(X_train, y_train)

# predict class labels for the test set
predicted = classifier_RF.predict(X_test)

# generate class probabilities
probs = classifier_RF.predict_proba(X_test)


# generate evaluation metrics

print("\t%s: %r" % ("accuracy_score is: ", accuracy_score(y_test, predicted)))
# print("\t%s: %r" % ("roc_auc_score is: ", roc_auc_score((y_test == '1').astype(int), probs[:, 1])))#string to int
# print("\t%s: %r" % ("f1_score is: ", f1_score((y_test == '1').astype(int), (predicted == '1').astype(int) )))#string to int
print("\t%s: %r" % ("roc_auc_score is: ", roc_auc_score(y_test, probs[:, 1])))
print("\t%s: %r" % ("f1_score is: ", f1_score(y_test, predicted )))#string to int

print ("confusion_matrix is: ")
cm = confusion_matrix(y_test, predicted)
print cm
print 'recall or sens =',float(cm[1,1])/(cm[1,0]+cm[1,1])
print 'fp =', float(cm[0,1])/cm[1,1]
print 'precision =', float(cm[1,1])/(cm[1,1] + cm[0,1])#1.0

/anaconda2/lib/python2.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


	accuracy_score is: : 0.9952305246422893
	roc_auc_score is: : 0.7940975693718695
	f1_score is: : 0.7013574660633484
confusion_matrix is: 
[[27389     0]
 [  132   155]]
sens = 0.540069686411
fp = 0.0


In [63]:
print 'precision =', float(cm[1,1])/(cm[1,1] + cm[0,1])

precision = 1.0


In [50]:
#SMOTE
#https://beckernick.github.io/oversampling-modeling/
# Q: what's next? smote first,or split
#Wrong way to sampling: smote before split train/test, then test data does not reflect true distribution in reality, 
#and “bleed” information from the test set into the training of the model. overfit! think about the case of simple 
#oversampling (where I just duplicate observations). If I upsample a dataset before splitting it into a train and 
#validation set, I could end up with the same observation in both datasets

#https://imbalanced-learn.org/en/stable/install.html
# Prerequisites
# The imbalanced-learn package requires the following dependencies:

# numpy (>=1.8.2)
# scipy (>=0.13.3)
# scikit-learn (>=0.20)
# keras 2 (optional)
# tensorflow (optional)
# Our release policy is to follow the scikit-learn releases in order to synchronize the new feature. imbalanced-learn 0.4 is the last version to support Python 2.7

# Install
# imbalanced-learn is currently available on the PyPi’s reporitories and you can install it via pip:

# pip install -U imbalanced-learn

#oversampling on only the training data, the right way!
sm = SMOTE(random_state=12)
x_train_sm, y_train_sm = sm.fit_sample(X_train, y_train)


unique, counts = np.unique(y_train_sm, return_counts=True)

print np.asarray((unique, counts)).T

[[     0 109572]
 [     1 109572]]


In [52]:

classifier_RF_sm = RandomForestClassifier(random_state=0, max_depth=3)

classifier_RF_sm.fit(x_train_sm, y_train_sm)

# predict class labels for the test set
predicted_sm = classifier_RF_sm.predict(X_test)

# generate class probabilities
probs_sm = classifier_RF_sm.predict_proba(X_test)


# generate evaluation metrics

print("\t%s: %r" % ("accuracy_score_sm is: ", accuracy_score(y_test, predicted_sm)))
# print("\t%s: %r" % ("roc_auc_score is: ", roc_auc_score((y_test == '1').astype(int), probs[:, 1])))#string to int
# print("\t%s: %r" % ("f1_score is: ", f1_score((y_test == '1').astype(int), (predicted == '1').astype(int) )))#string to int
print("\t%s: %r" % ("roc_auc_score_sm is: ", roc_auc_score(y_test, probs_sm[:, 1])))
print("\t%s: %r" % ("f1_score_sm is: ", f1_score(y_test, predicted_sm )))#string to int

print ("confusion_matrix_sm is: ")
cm_sm = confusion_matrix(y_test, predicted_sm)
print cm_sm
print 'recall or sens_sm =',float(cm_sm[1,1])/(cm_sm[1,0]+cm_sm[1,1])
print 'fp_sm =', float(cm_sm[0,1])/cm_sm[1,1]
print 'precision_sm =', float(cm_sm[1,1])/(cm_sm[1,1] + cm_sm[0,1])



	accuracy_score_sm is: : 0.9883292383292384
	roc_auc_score_sm is: : 0.7939583313985891
	f1_score_sm is: : 0.4516129032258065
confusion_matrix_sm is: 
[[27220   169]
 [  154   133]]
sens_sm = 0.463414634146
fp_sm = 1.27067669173


In [28]:
classifier_RF

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [ ]:
	accuracy_score is: : 0.9958447752565399
	roc_auc_score is: : 0.8101936444639452
	f1_score is: : 0.7494553376906318
confusion_matrix is: 
[[27389     0]
 [  115   172]]
sens = 0.599303135889
fp = 0.0



In [22]:
np.unique(predicted)

array([0, 1])

In [24]:

unique, counts = np.unique(predicted, return_counts=True)

print np.asarray((unique, counts)).T

[[    0 27504]
 [    1   172]]


In [25]:
unique, counts = np.unique(y_test, return_counts=True)

print np.asarray((unique, counts)).T

[[    0 27389]
 [    1   287]]


In [57]:
# from sklearn.grid_search import GridSearchCV
#https://stackoverflow.com/questions/40615021/cannot-import-sklearn-model-selection-in-scikit-learn
import sklearn
print(sklearn.__version__)
from sklearn.model_selection import GridSearchCV
def print_grid_search_metrics(gs):
    print "Best score: %0.3f" % gs.best_score_
    print "Best parameters set:"
    best_parameters = gs.best_params_
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))

0.20.2


In [ ]:
do logistic, partial auc,precision
discuss the eval metric


In [ ]:
#Optimized by f1

In [58]:
parameters = {        
'max_depth': [5, 15, 25],
'n_estimators' :  [50,150,250],
'class_weight' : [{0: 1, 1: w} for w in [1, 50, 100]]
}

Grid_RF_f1 = GridSearchCV(RandomForestClassifier(random_state=0),parameters,n_jobs=-1,
                       verbose=2, scoring='f1', cv=3)
Grid_RF_f1.fit(X_train, y_train)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  81 out of  81 | elapsed:  4.7min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [50, 150, 250], 'max_depth': [5, 15, 25], 'class_weight': [{0: 1, 1: 1}, {0: 1, 1: 50}, {0: 1, 1: 100}]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1', verbose=2)

In [59]:
print_grid_search_metrics(Grid_RF_f1)

# Best score: 0.737
# Best parameters set:
# 	class_weight: {0: 1, 1: 1}
# 	max_depth: 15
# 	n_estimators: 150

Best score: 0.730
Best parameters set:
	class_weight: {0: 1, 1: 1}
	max_depth: 15
	n_estimators: 250


In [60]:
best_RF_model_f1 = Grid_RF_f1.best_estimator_
best_RF_model_f1

RandomForestClassifier(bootstrap=True, class_weight={0: 1, 1: 1},
            criterion='gini', max_depth=15, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=250, n_jobs=None, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [62]:
pd.DataFrame(best_RF_model_f1.feature_importances_, index = X_train.columns, columns=['importance']).sort_values('importance', ascending=False)

,importance
interval_after_signup,0.398976
purchase_days_of_year,0.198068
n_ip_shared,0.123500
n_dev_shared,0.078777
signup_days_of_year,0.046650
purchase_seconds_of_day,0.038771
signup_seconds_of_day,0.037961
purchase_value,0.021785
age,0.017747
n_country_shared,0.014554


In [64]:


# predict class labels for the test set
predictedBest_f1 = best_RF_model_f1.predict(X_test)

# generate class probabilities
probsBest_f1 = best_RF_model_f1.predict_proba(X_test)


# generate evaluation metrics

print("\t%s: %r" % ("best accuracy_score_f1 is: ", accuracy_score(y_test, predictedBest_f1)))
print("\t%s: %r" % ("best roc_auc_score_f1 is: ", roc_auc_score(y_test, probsBest_f1[:, 1])))
print("\t%s: %r" % ("best f1_score_f1 is: ", f1_score(y_test, predictedBest_f1 )))




print ("best confusion_matrix_f1 is: ")
bestcm_f1 = confusion_matrix(y_test, predictedBest_f1)
print bestcm_f1
print 'best recall or sens_f1 =',float(bestcm_f1[1,1])/(bestcm_f1[1,0]+bestcm_f1[1,1])
print 'best fp_f1 =', float(bestcm_f1[0,1])/bestcm_f1[1,1]
print 'best precision_f1 =', float(bestcm_f1[1,1])/(bestcm_f1[1,1] + bestcm_f1[0,1])

	best accuracy_score_f1 is: : 0.9954111865876571
	best roc_auc_score_f1 is: : 0.8094087213984912
	best f1_score_f1 is: : 0.7158836689038032
best confusion_matrix_f1 is: 
[[27389     0]
 [  127   160]]
best sens_f1 = 0.557491289199
best fp_f1 = 0.0
best precision_f1 = 1.0


In [ ]:
#Optimized by recall

In [65]:


Grid_RFz_recall = GridSearchCV(RandomForestClassifier(random_state=0),parameters,n_jobs=-1,
                       verbose=2, scoring='recall', cv=3)
Grid_RFz_recall.fit(X_train, y_train)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  81 out of  81 | elapsed:  4.6min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [50, 150, 250], 'max_depth': [5, 15, 25], 'class_weight': [{0: 1, 1: 1}, {0: 1, 1: 50}, {0: 1, 1: 100}]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='recall', verbose=2)

In [66]:
print_grid_search_metrics(Grid_RFz_recall)


Best score: 0.662
Best parameters set:
	class_weight: {0: 1, 1: 100}
	max_depth: 5
	n_estimators: 150


In [67]:
best_RF_model_recall = Grid_RFz_recall.best_estimator_
best_RF_model_recall

RandomForestClassifier(bootstrap=True, class_weight={0: 1, 1: 100},
            criterion='gini', max_depth=5, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=150, n_jobs=None, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [68]:
# predict class labels for the test set
predictedBest_recall = best_RF_model_recall.predict(X_test)

# generate class probabilities
probsBest_recall = best_RF_model_recall.predict_proba(X_test)


# generate evaluation metrics

print("\t%s: %r" % ("best accuracy_score_recall is: ", accuracy_score(y_test, predictedBest_recall)))
print("\t%s: %r" % ("best roc_auc_score_recall is: ", roc_auc_score(y_test, probsBest_recall[:, 1])))
print("\t%s: %r" % ("best f1_score_recall is: ", f1_score(y_test, predictedBest_recall )))




print ("best confusion_matrix by recall is: ")
bestcm_recall = confusion_matrix(y_test, predictedBest_recall)
print bestcm_recall
print 'best recall optimized by recall =',float(bestcm_recall[1,1])/(bestcm_recall[1,0]+bestcm_recall[1,1])
print 'best precision optimized by recall =', float(bestcm_recall[1,1])/(bestcm_recall[1,1] + bestcm_recall[0,1])

	best accuracy_score_recall is: : 0.9864503540974129
	best roc_auc_score_recall is: : 0.8152250394783226
	best f1_score_recall is: : 0.4665718349928876
best confusion_matrix_recall is: 
[[27137   252]
 [  123   164]]
best sens_recall = 0.571428571429
best precision_recall = 0.394230769231


In [72]:
pd.DataFrame(best_RF_model_recall.feature_importances_, index = X_train.columns, columns=['importance']).sort_values('importance', ascending=False)

,importance
n_dev_shared,0.310133
interval_after_signup,0.249985
purchase_days_of_year,0.195365
signup_days_of_year,0.106627
n_ip_shared,0.103732
purchase_seconds_of_day,0.007357
signup_seconds_of_day,0.007107
age,0.005625
purchase_value,0.005614
n_country_shared,0.003221


In [96]:
trainDF = pd.concat([X_train, y_train], axis=1)
pd.crosstab(trainDF["n_dev_shared"],trainDF["class"])

class,0,1
n_dev_shared,,
0.0,104960,413
0.2,4397,359
0.4,167,208
0.6,35,97
0.8,9,31
1.0,4,20


In [77]:
fraudDF.groupby("class")[['interval_after_signup']].mean()
# 

,interval_after_signup
class,
0,5.191179e+06
1,2.354059e+06


In [ ]:
fraudDF.groupby("class")[['interval_after_signup']].median()
#half of fraud happened 1 s after signed up

In [85]:
# fraud_data[(fraud_data.class == 1)].head(100)
fraud_data[fraud_data['class'] == 1].head(100)

,user_id,signup_time,purchase_time,purchase_value,device_id,sex,age,ip_address,class,country,interval_after_signup,signup_days_of_year,signup_seconds_of_day,purchase_days_of_year,purchase_seconds_of_day,source_Ads,source_Direct,source_SEO,browser_Chrome,browser_FireFox,browser_IE,browser_Opera,browser_Safari
136961,86574,2015-01-11 14:39:04,2015-01-11 14:39:05,61,ZQVCVEXPSAZCA,1,23,3.941890e+08,1,United States,1.0,11,52744,11,52745,0,0,1,0,0,0,0,1
136962,74145,2015-03-27 07:55:56,2015-07-20 06:59:34,55,DFQZESNZDEIPP,0,18,1.930175e+09,1,Korea Republic of,9932618.0,86,28556,201,25174,1,0,0,0,1,0,0,0
136963,282058,2015-01-05 08:18:00,2015-01-05 08:18:01,36,EGLGSEGYPMMAM,1,37,3.503224e+09,1,United States,1.0,5,29880,5,29881,0,0,1,0,1,0,0,0
136964,255332,2015-01-01 01:46:58,2015-01-01 01:46:59,19,SQWWBFDRYPGYA,0,36,2.105295e+08,1,United States,1.0,1,6418,1,6419,0,0,1,0,0,1,0,0
136965,399497,2015-01-01 14:29:27,2015-01-22 15:14:12,52,JWAVUHXQXTCHW,1,22,1.052881e+09,1,Nigeria,1817085.0,1,52167,22,54852,0,1,0,1,0,0,0,0
136966,337297,2015-02-10 13:01:00,2015-04-14 05:08:57,50,BCKHXUUTHTMMW,0,31,8.000143e+08,1,Canada,5414877.0,41,46860,104,18537,0,0,1,0,1,0,0,0
136967,46163,2015-03-05 09:23:48,2015-06-03 01:57:00,12,YFFPJJLHEHZML,1,45,1.957369e+09,1,China,7749192.0,64,33828,154,7020,0,1,0,0,0,0,0,1
136968,17097,2015-01-08 10:34:16,2015-01-08 10:34:17,12,IGAXAVAZFJYOB,1,33,2.447058e+09,1,Netherlands,1.0,8,38056,8,38057,1,0,0,0,0,0,0,1
136969,318808,2015-01-02 06:01:09,2015-01-02 06:01:10,22,MLKPWLXKZYANO,0,33,2.543532e+09,1,United States,1.0,2,21669,2,21670,1,0,0,1,0,0,0,0
136970,303431,2015-01-11 19:16:05,2015-01-11 19:16:06,61,SIKJNYXDSEUEG,1,30,3.410478e+09,1,Taiwan; Republic of China (ROC),1.0,11,69365,11,69366,1,0,0,1,0,0,0,0


In [98]:
probsBest_recall[:, 1]

array([0.27621914, 0.24952888, 0.28019583, ..., 0.29030727, 0.27723496,
       0.25126635])

In [102]:
t = (10 * probsBest_recall[:, 1]).astype(int)
unique, counts = np.unique(t, return_counts=True)

print np.asarray((unique, counts)).T

[[    1     2]
 [    2 26975]
 [    3   202]
 [    4    81]
 [    5   248]
 [    6     8]
 [    7     6]
 [    8    39]
 [    9   115]]


In [81]:
fraud_data[(fraud_data.device_id == 'YSSKYOSJHPPLJ')].head(100)

,user_id,signup_time,purchase_time,purchase_value,device_id,sex,age,ip_address,class,country,interval_after_signup,signup_days_of_year,signup_seconds_of_day,purchase_days_of_year,purchase_seconds_of_day,source_Ads,source_Direct,source_SEO,browser_Chrome,browser_FireFox,browser_IE,browser_Opera,browser_Safari
69666,273863,2015-01-01 18:52:33,2015-01-31 07:47:31,15,YSSKYOSJHPPLJ,1,53,2.621474e+09,0,United States,2552098.0,1,67953,31,28051,0,0,1,0,0,0,1,0
137021,260959,2015-01-01 18:52:39,2015-01-01 18:52:40,15,YSSKYOSJHPPLJ,1,53,2.621474e+09,1,United States,1.0,1,67959,1,67960,0,0,1,0,0,0,1,0


In [16]:
fraud_data.to_csv("fraud_cleaned.csv", index=False)


In [20]:
def split_train_set(df):
    X = df.loc[:, df.columns != 'is_fraud']
    y = df.is_fraud

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    print("X_train.shape:", X_train.shape)
    print("y_train.shape:", y_train.shape)

    split = int(len(y_test) / 2)
    X_valid = X_test[:split]
    y_valid = y_test[:split]
    X_test = X_test[split:]
    y_test = y_test[split:]
    print("X_valid.shape:", X_valid.shape)
    print("y_valid.shape:", y_valid.shape)
    print("X_test.shape:", X_test.shape)
    print("y_test.shape:", y_test.shape)

    return X_train, y_train, X_valid, y_valid, X_test, y_test


def train_xgboost_model(df):
    X_train, y_train, X_valid, y_valid, X_test, y_test = split_train_set(df)

    total_neg = df[df.is_fraud == 0].shape[0]
    total_pos = df[df.is_fraud == 1].shape[0]
    print('neg total:', total_neg)
    print('pos total:', total_pos)

    # ======
    booster_params = {
        'silent': 1,
        'max_depth': 6,
        'eta': 0.1,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'objective': 'binary:logistic',  # output probabilities
        'eval_metric': ['auc'],
    }

    learning_task_params = {
        'num_boost_round': 300,  # xgb.XGBRegressor takes n_estimators
        'early_stopping_rounds': 30,
        'nfold': 5,
        'verbose_eval': 10,
        'seed': 37,
    }

    # booster_params.update(learning_task_params)
    # ======

    train_matrix = xgb.DMatrix(X_train, y_train)
    valid_matrix = xgb.DMatrix(X_valid, y_valid)
    test_matrix = xgb.DMatrix(X_test, y_test)

    # pick best params
    cv_results = xgb.cv(booster_params, train_matrix,
                        num_boost_round=learning_task_params['num_boost_round'],
                        nfold=learning_task_params['nfold'],
                        metrics=booster_params['eval_metric'],
                        early_stopping_rounds=learning_task_params["early_stopping_rounds"],
                        verbose_eval=learning_task_params['verbose_eval'],
                        seed=learning_task_params['seed'])
    # print(cv_results)
    n_best_trees = cv_results.shape[0]
    print('\nn_best_trees', n_best_trees, '\n')

    learning_progress = dict()
    watchlist = [(train_matrix, 'train'), (valid_matrix, 'valid')]
    gbt_model = xgb.train(booster_params, train_matrix,
                          num_boost_round=n_best_trees,
                          evals=watchlist,
                          evals_result=learning_progress)

    print("\nModel Report")
    plot_importance(gbt_model)
    plt.show()

    # print(learning_progress)
    plot_learning_curves(learning_progress)

    # make predictions for validation data
    y_pred_valid = gbt_model.predict(valid_matrix, ntree_limit=n_best_trees)
    evaluate_predictions(y_valid, y_pred_valid, 'Valid')

    # Making the Confusion Matrix
    cm = metrics.confusion_matrix(y_valid, (y_pred_valid > 0.5))
#     print(colored('The Confusion Matrix on Valid Data is:', 'red'), '\n', colored(cm, 'red'))
    print(cm)
    print()

    plot_roc(y_valid, y_pred_valid)

    # make predictions for test data
    y_pred_test = gbt_model.predict(test_matrix, ntree_limit=n_best_trees)
    evaluate_predictions(y_test, y_pred_test, 'Test')

    # plot_tree(gbt_model)
    # plt.show()


def evaluate_predictions(y_true, y_pred, status='Train'):
    predictions = [round(value) for value in y_pred]
    # evaluate predictions
    print("Accuracy : %.4g" % metrics.accuracy_score(y_true, predictions))
    print("AUC Score (%s): %f" % (status, metrics.roc_auc_score(y_true, y_pred)))


def plot_roc(y_true, y_pred):
    fpr, tpr, thresholds = metrics.roc_curve(y_true, y_pred)
    roc = pd.DataFrame({'FPR': fpr, 'TPR': tpr, 'Threshold': thresholds})

    plt.figure(figsize=(10, 5))
    plt.plot(roc.FPR, roc.TPR, marker='h')
    plt.title('XGBoost ROC')
    plt.xlabel("FPR")
    plt.ylabel("TPR")


def plot_learning_curves(progress):
    # retrieve performance metrics
    epochs = len(progress['train']['auc'])
    x_axis = range(0, epochs)

    # plot auc
    fig, ax = plt.subplots()
    ax.plot(x_axis, progress['train']['auc'], label='Train')
    ax.plot(x_axis, progress['valid']['auc'], label='Valid')
    ax.legend()
    plt.xlabel('num_boost_round')
    plt.ylabel('Auc')
    plt.title('XGBoost Auc')
    plt.show()

    # # plot classification error
    # fig, ax = plt.subplots()
    # ax.plot(x_axis, progress['train']['error'], label='Train')
    # ax.plot(x_axis, progress['valid']['error'], label='Valid')
    # ax.legend()
    # plt.ylabel('Classification Error')
    # plt.title('XGBoost Classification Error')
    # plt.show()

In [ ]:
#HW: try xgboost
Q3 var imp freq table
Q4 buckets red yellow green